In [246]:
%run model.py

In [247]:
BASE_DIR = os.path.abspath('.')
BATCH_SIZE = 4
# modelnet_dataset = ModelNetDataset(root=BASE_DIR)
# modelnet_dataloader = DataLoader(modelnet_dataset, batch_size=BATCH_SIZE)
# for (modelnet_pc, modelnet_label) in modelnet_dataloader:
#     break

s3dis_dataset = S3DISDataset(root=BASE_DIR)

s3dis_dataloader = DataLoader(s3dis_dataset, batch_size=BATCH_SIZE)
for (s3dis_pc, s3dis_label) in s3dis_dataloader:
    break
pc_dataset = s3dis_dataset

point_feature = PointFeature(num_point=pc_dataset.num_points,
                                global_feature=False)
cls_net = PointNetCls(num_class=pc_dataset.num_classes,
                        num_point=pc_dataset.num_points)
seg_net = PointNetSeg(num_class=pc_dataset.num_classes,
                        num_point=pc_dataset.num_points)
segfc_net = PointNetSegFC(num_class=pc_dataset.num_classes,
                            num_point=pc_dataset.num_points)


# feature = point_feature(modelnet_pc)

# seg_fc = segfc_net(modelnet_pc)

# output = cls_net(modelnet_pc)
# print(output.shape)
# pred_num = output.argmax(dim=1)
# for i in range(4):
#     print(f'Item {i+1}: {modelnet_dataset.class_names[pred_num[i]]}')

# print(pred_num.shape, modelnet_label.shape)

# correct = (pred_num == modelnet_label.squeeze()).sum().item()
# print(correct)

# seg = seg_net(modelnet_pc)
# print(seg.shape)


In [248]:
import torch.optim as optim

optimizer = optim.Adam(seg_net.parameters(), lr=0.0001)
i=0
while True:
    optimizer.zero_grad()
    seg = seg_net(s3dis_pc)
    loss = F.nll_loss(seg.view(-1, 13), s3dis_label.view(-1).long())
    print(loss)
    loss.backward()
    optimizer.step()
    i+=1
    if i >2:
        break

tensor(2.5847, grad_fn=<NllLossBackward>)
tensor(2.2747, grad_fn=<NllLossBackward>)
tensor(2.0795, grad_fn=<NllLossBackward>)


In [249]:
import torch.utils.data as data
from dataset import ModelNetDataset, S3DISDataset
import os

BASE_DIR = os.path.dirname('.')

BATCH_SIZE = 4
WORKERS = 0

dataset = S3DISDataset(root=BASE_DIR)
test_dataset = S3DISDataset(root=BASE_DIR, train=False)

dataloader = data.DataLoader(dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=WORKERS)

NUM_CLASSES = dataset.num_classes
NUM_POINTS = dataset.num_points

test_dataloader = data.DataLoader(test_dataset,
                                   batch_size=BATCH_SIZE,
                                   shuffle=True,
                                   num_workers=WORKERS)

print(f'Length of training dataset: {len(dataset)}')
print(f'Length of testing dataset: {len(test_dataset)}')
num_classes = dataset.num_classes
print(f'Number of classes: {num_classes}')

Length of training dataset: 19898
Length of testing dataset: 3687
Number of classes: 13


In [250]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from model import PointNetCls, PointNetSeg

net = PointNetSeg(num_class=NUM_CLASSES, num_point=NUM_POINTS)

optimizer = optim.Adam(net.parameters(), lr=0.001)


num_batch = int(len(dataset) / BATCH_SIZE) \
    if len(dataset) % BATCH_SIZE == 0 \
    else int(len(dataset) / BATCH_SIZE)+1
print(f'Number of batches: {num_batch}')

for epoch in range(2):
    for i, (point_cloud, label) in enumerate(dataloader):
        label = label.view(-1).long()
        if torch.cuda.is_available():
            point_cloud, label = point_cloud.cuda(), label.cuda()
        optimizer.zero_grad()
        net = net.train()
        pred = net(point_cloud)
        pred = pred.view(-1, NUM_CLASSES)
        loss = F.nll_loss(pred, label)
        loss.backward()
        optimizer.step()
        pred_num = pred.argmax(dim=1)
        correct = (pred_num == label).sum().item()
        print(f'Epoch: {epoch}, iter: {i}, train loss: {loss.item():.3f}, \
accuracy:{100*correct/(BATCH_SIZE*NUM_POINTS):.3f}%')

        if i % 10 == 9:
            j, (test_points, test_labels) = next(enumerate(test_dataloader))
            test_labels = test_labels.view(-1).long()
            if torch.cuda.is_available():
                test_points, test_labels = test_points.cuda(
                ), test_labels.cuda()
            net = net.eval()
            pred = net(test_points)
            pred = pred.view(-1, NUM_CLASSES)
            loss = F.nll_loss(pred, test_labels)
            pred_num = pred.argmax(dim=1)
            correct = (pred_num == test_labels).sum().item()
            print(
                f'\nEpoch: {epoch}, iter: {i}, test loss: {loss.item():.3f}, \
accuracy:{100*correct/(BATCH_SIZE*NUM_POINTS):.3f}%\n')

Number of batches: 4975
Epoch: 0, iter: 0, train loss: 2.613, accuracy:10.059%
Epoch: 0, iter: 1, train loss: 2.517, accuracy:13.843%
Epoch: 0, iter: 2, train loss: 2.141, accuracy:36.572%
Epoch: 0, iter: 3, train loss: 2.169, accuracy:36.292%
Epoch: 0, iter: 4, train loss: 2.153, accuracy:33.331%
Epoch: 0, iter: 5, train loss: 2.109, accuracy:39.001%
Epoch: 0, iter: 6, train loss: 1.869, accuracy:52.997%
Epoch: 0, iter: 7, train loss: 2.137, accuracy:33.508%
Epoch: 0, iter: 8, train loss: 1.888, accuracy:47.296%
Epoch: 0, iter: 9, train loss: 1.933, accuracy:48.126%

Epoch: 0, iter: 9, test loss: 2.323, accuracy:14.154%

Epoch: 0, iter: 10, train loss: 1.820, accuracy:48.828%
Epoch: 0, iter: 11, train loss: 1.945, accuracy:37.354%
Epoch: 0, iter: 12, train loss: 1.658, accuracy:49.774%
Epoch: 0, iter: 13, train loss: 1.159, accuracy:78.772%
Epoch: 0, iter: 14, train loss: 1.643, accuracy:55.438%


KeyboardInterrupt: 

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def square_distance(src, dst):
    '''
    Input:
        src: [B, N, C]
        dst: [B, M, C]
    Retrun:
        dist: [B, N, M]
    '''

    B, N, C = src.shape
    _, M, _ = dst.shape

    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src**2, dim=-1).view(B, N, 1)
    dist += torch.sum(dst**2, dim=-1).view(B, 1, M)

    return dist


def farthest_point_sample(xyz, npoint):
    '''
    Input:
        xyz: [B, N, C]
        npoint: Number of point for centered points
    Return:
        centered_idx: [B, npoint]
    '''
    B, N, C = xyz.shape
    # [B, npoint]
    centered_idx = torch.zeros(B, npoint, dtype=torch.long)
    # [B]
    farthest = torch.randint(0, N, (B, ), dtype=torch.long)
    # [B, N]
    distance = torch.ones(B, N) * 1e10
    # [B]
    batch_indices = torch.arange(0, B, dtype=torch.long)
    for i in range(npoint):
        centered_idx[:, i] = farthest
        centered_xyz = xyz[batch_indices, farthest, :].view(B, 1, C)
        dist = torch.sum((xyz - centered_xyz)**2, dim=-1)
        # take the shortest distance as the distance from the remaining point
        # set to the selected point set
        # (if the point has been selected, the distance=0 is not updated)
        mask = dist < distance
        distance[mask] = dist[mask]
        # update the farthest point by taking the index of the longest
        # distance from the remaining point set to the selected point set
        farthest = distance.max(-1)[1]

    return centered_idx


def index_points(points, idx):
    '''
    Input:
        points: [B, N, D]
        idx: [B, npoint, nsample]
    Return:
        new_points: [B, npoint, nsample, D]
    '''
    B, N, D = points.shape
    view_list = list(idx.shape)
    view_list[1:] = [1] * (len(view_list) - 1)
    repeat_list = list(idx.shape)
    repeat_list[0] = 1
    batch_indices = torch.arange(B).view(view_list).repeat(repeat_list)
    new_points = points[batch_indices, idx, :]

    return new_points


def query_ball_point(nsample, radius, xyz, centered_xyz):
    '''
    Input:
        nsample: Number of point for each centered point/ball query
        radius: Radius of the local query ball region
        xyz: position data [B, N, C]
        centered_xyz: position data of query points [B, npoint, C]
    Return:
        grouped_idx: indices of neighbouring points [B, npoint, nsample]
    '''
    B, N, C = xyz.shape
    _, npoint, _ = centered_xyz.shape
    # [B, npoint, nsample]
    grouped_idx = torch.zeros(B, npoint, nsample, dtype=torch.long)
    # [B, npiont, N]
    distance = square_distance(centered_xyz, xyz)
    # [B, npoint, N]
    mask_N = distance > radius**2
    # [B, npoint, N], assign a very large number (regarded as infinite) to all
    # points outside the local ball region
    distance[mask_N] = 1e10
    # [B, npoint, nsample]
    # take the first nsample closest points of each query(centered) point
    # the numnber of local points might be less than nsample
    grouped_idx = distance.sort(dim=-1)[1][..., :nsample]
    # [B, npoint, nsample]
    # duplicate the first point index
    grouped_first = grouped_idx[..., 0].view(B, npoint,
                                             1).repeat(1, 1, nsample)
    # [B, npoint, nsample]
    # mask the indices of points outside the local ball region
    mask_nsample = distance.sort(dim=-1)[0][..., :nsample] == 1e10
    # [B, npoint, nsample]
    # assign the first point index to all the outside points
    grouped_idx[mask_nsample] = grouped_first[mask_nsample]

    return grouped_idx


def sample_and_group(npoint, radius, nsample, points):
    '''
    Input:
        npoint: Number of point for centered points (by FPS sampling)
        radius: Radius of the local query ball region
        nsample: Number of point for each centered point/ball query
        points: position data and features [B, N, C+D]
    Return:
        centered_points: [B, npoint, C+D]
        grouped_points: [B, npoint, nsample, C+D]
    '''
    # [B, N, C]
    xyz = points[..., :3]
    # [B, N, D]
    features = points[..., 3:]
    # [B, npoint]
    centered_idx = farthest_point_sample(xyz, npoint)
    # [B, npoint, C]
    centered_xyz = index_points(xyz, centered_idx)
    # [B, npoint, D]
    centered_features = index_points(features, centered_idx)
    # [B, npoint, C+D]
    centered_points = torch.cat((centered_xyz, centered_features), dim=-1)

    # [B, npoint, nsample]
    grouped_idx = query_ball_point(nsample, radius, xyz, centered_xyz)
    # [B, npoint, nsample, C]
    grouped_xyz = index_points(xyz, grouped_idx)
    # [B, npoint, nsample, D]
    grouped_features = index_points(features, grouped_idx)
    # [B, npoint, nsample, C+D]
    grouped_points = torch.cat((grouped_xyz, grouped_features), dim=-1)

    return centered_points, grouped_points


def sample_and_group_all(points):
    '''
    This function takes all points as a group,
    i.e., npoint = 1 and nsample = N.

    Input:
        points: [B, N, C+D]
    Return:
        centered_points: [B, 1, C+D]
        grouped_points: [B, 1, N, C+D]
    '''
    B, N, _ = points.shape
    # [B]
    centered_idx = torch.randint(0, N, (B, ), dtype=torch.long)
    # [B, 1, C+D]
    centered_points = index_points(points, centered_idx).view(B, 1, -1)
    # [B, 1, N, C+D]
    grouped_points = points.view(B, 1, N, -1)

    return centered_points, grouped_points


class GraphAttention(nn.Module):
    def __init__(self, all_channel, feature_channel):
        super(GraphAttention, self).__init__()
        # initialize a matrix to compute attention scores of
        # each centered point and its neighbouring points
        self.a = nn.Parameter(torch.zeros(all_channel, feature_channel))

    def forward(self, centered_points, grouped_points):
        '''
        Input:
            centered_points: [B, npoint, C+D]
            grouped_points: [B, npoint, nsample, C+D]
        Return:
            centered_features: [B, npoint, D]
        '''
        B, npoint, nsample, dim = grouped_points.shape
        # [B, npoint, nsample, D]
        grouped_features = grouped_points[..., 3:]
        # [B, npoint, nsample, C+D]
        # compute the position and feature difference between
        # the centered point and its neighbouring points
        delta_p_concat_delta_h = grouped_points - centered_points.view(
            B, npoint, 1, dim)
        # [B, npoint, nsample, D]
        attention = torch.matmul(delta_p_concat_delta_h, self.a)
        # [B, npoint, nsample, D]
        # normalize along each centered point's nsample neighbouring points
        attention = attention.softmax(dim=2)
        # [B, npoint, nsample, D]
        # return weighted sum of each centered points by element-wise product
        # of the computed attention scores and its responding neighbouring
        # points' features, the output works as the centered_features for the
        # next layer in GAC_Conv_layer
        centered_features = torch.sum(torch.mul(attention, grouped_features),
                                      dim=2)

        return centered_features


class GraphAttentionConvLayer(nn.Module):
    def __init__(self, npoint, radius, nsample, mlp, feature_channel):
        '''
        Input:
                npoint: Number of point for centered points (by FPS sampling)
                radius: Radius of the local query ball region
                nsample: Number of point for each centered point/ball query
                mlp: A list for mlp input-output channel, e.g., [64, 64, 128]
                feature_channel: the dimention of the input feature channel
        '''
        super(GraphAttentionConvLayer, self).__init__()
        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.conv2ds = nn.ModuleList()
        self.bns = nn.ModuleList()
        last_channel = feature_channel
        for out_channel in mlp:
            self.conv2ds.append(nn.Conv2d(last_channel, out_channel, 1))
            self.bns.append(nn.BatchNorm2d(out_channel))
            last_channel = out_channel

        self.GAC = GraphAttention(3 + last_channel, last_channel)

    def forward(self, points):
        '''
        Input:
            points: [B, N, C+D]
        Return:
            new_points: the centered points with position data and
                        aggregated features from input points [B, npoint, C+D']
        '''
        B, N, _ = points.shape
        # [B, npoint, C+D]
        # [B, npoint, nsample, C+D]
        centered_points, grouped_points = sample_and_group(
            self.npoint, self.radius, self.nsample, points)
        # [B, npoint, C]
        centered_xyz = centered_points[..., :3]
        # [B, D, 1, npoint]
        # permute the dimensions for conv2d operation
        centered_features = centered_points[..., 3:].view(
            B, self.npoint, 1, -1).permute(0, 3, 2, 1)
        # [B, npoint, nsample, C]
        grouped_xyz = grouped_points[..., :3]
        # [B, D, nsample, npoint]
        grouped_features = grouped_points[..., 3:].permute(0, 3, 2, 1)

        # map centered and grouped features into high-level ones,
        # which have D' feature channels after the operation
        for i, conv2d in enumerate(self.conv2ds):
            bn = self.bns[i]
            centered_features = bn(conv2d(centered_features))
            grouped_features = bn(conv2d(grouped_features))

        # [B, npoint, D']
        centered_features = centered_features.permute(0, 3, 2, 1).squeeze()
        # [B, npoint, nsample, D']
        grouped_features = grouped_features.permute(0, 3, 2, 1)
        # [B, npoint, C+D']
        centered_points = torch.cat((centered_xyz, centered_features), dim=-1)
        # [B, npoint, nsample, C+D']
        grouped_points = torch.cat((grouped_xyz, grouped_features), dim=-1)
        # [B, npoint, D']
        # compute the aggregated features of each centered point
        gac_features = self.GAC(centered_points, grouped_points)
        # [B, npoint, C+D']
        new_points = torch.cat((centered_xyz, gac_features), dim=-1)

        return new_points


class FeaturePropagationLayer(nn.Module):
    def __init__(self, in_channel, mlp):
        super(FeaturePropagationLayer, self).__init__()
        self.conv1ds = nn.ModuleList()
        self.bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.conv1ds.append(nn.Conv1d(last_channel, out_channel, 1))
            self.bns.append(nn.BatchNorm1d(out_channel))
            last_channel = out_channel

    def forward(self, layer1_points, layer2_points):
        '''
        Input:
            points_1: input of a GAC_Layer [B, N, C + D_1]
            points_2: output of the corresponding GAC_Layer
                        [B, npoint, C + D_2]
            N_1 >= N_2
        Return:
            new_points: [B, N, C + D_3]
        '''
        B, N, dim = layer1_points.shape
        _, npoint, _ = layer2_points.shape
        # [B, N, C]
        layer1_xyz = layer1_points[..., :3]
        # [B, N, D_1]
        layer1_features = layer1_points[..., 3:]
        # [B, npoint, C]
        layer2_xyz = layer2_points[..., :3]
        # [B, npoint, D_2]
        layer2_features = layer2_points[..., 3:]
        # if there is only one sampled point, copy the features of this point as the interpolated features
        if npoint == 1:
            # [B, N, D_2]
            interpolated_features = layer2_features.repeat(1, N, 1)
        # [B, N, npoint]
        distance = square_distance(layer1_xyz, layer2_xyz)
        # [B, N, npoint]
        dist, idx = distance.sort(-1)
        # [B, N, 3]
        dist_knn = dist[..., :3]
        # avoid dividing by 0
        dist_knn[dist_knn < 1e10] = 1e-10
        idx_knn = idx[..., :3]
        # [B, N, 3]
        weight = 1 / dist_knn
        # [B, N, 3, D_2]
        layer2_features_knn = index_points(layer2_features, idx_knn)
        # [B, N, D_2]
        interpolated_features = torch.sum(
            torch.mul(weight.view(B, N, 3, 1), layer2_features_knn),
            dim=-2) / torch.sum(weight, dim=-1).view(B, N, 1)
        if dim == 3:
            # [B, N, D_2]
            new_features = interpolated_features
        else:
            # [B, N, D_1 + D_2]
            new_features = torch.cat((layer1_features, interpolated_features), dim=-1)
        new_features = new_features.permute(0, 2, 1)
        for i, conv1d in enumerate(self.conv1ds):
            bn = self.bns[i]
            new_features = F.relu(bn(conv1d(new_features)))
        # [B, N, C + D_3]
        new_points = torch.cat((layer1_xyz, new_features.permute(0, 2, 1)), dim=-1)

        return new_points


In [15]:
src = torch.ones(4, 5, 3, dtype=torch.long)
dst = torch.arange(1, 25, dtype=torch.long).view(4,6,1).repeat(1,1,3)
torch.manual_seed(4*4096*(3+6))
xyz = torch.randn(4,4096,3)
features = torch.randn(4,4096,6)
points = torch.cat((xyz, features), dim=-1)
npoint = 100
nsample = 8
radius = 2

square_dist = square_distance(src, dst)
centered_idx = farthest_point_sample(xyz, npoint)
centered_xyz = index_points(xyz, centered_idx)
grouped_idx = query_ball_point(nsample, radius, xyz, centered_xyz)
grouped_xyz = index_points(xyz, grouped_idx)
centered_points, grouped_points = sample_and_group(npoint, radius, nsample, points)
grouped_points.shape

torch.Size([4, 100, 8, 9])

In [16]:
layer1 = GraphAttentionConvLayer(npoint, radius, nsample, [16, 32], 6)
layer2 = GraphAttentionConvLayer(int(npoint/4), radius, nsample, [32, 64], 32)
layer1_points = layer1(points)
layer2_points = layer2(layer1_points)
print(f'layer1_points: {layer1_points.shape}\nlayer2_points: {layer2_points.shape}')

layer1_points: torch.Size([4, 100, 35])
layer2_points: torch.Size([4, 25, 67])


In [17]:
fp1 = FeaturePropagationLayer(96, [128, 256])
l1_point = fp1(layer1_points, layer2_points)
l1_point.shape

torch.Size([4, 100, 259])

In [78]:
class GAC_Net(nn.Module):
    def __init__(self, num_classes):
        super(GAC_Net, self).__init__()
        self.num_classes = num_classes
        # GraphAttentionConvLayer(npoint, radius, nsample, mlp, feature_channel)
        # [B, 1024, 32+3]
        self.graph_pooling1 = GraphAttentionConvLayer(1024, 2, 32, [16, 32], 6)
        # [B, 256, 128+3]
        self.graph_pooling2 = GraphAttentionConvLayer(256, 2, 32, [64, 128], 32)
        # [B, 64, 256+3]
        self.graph_pooling3 = GraphAttentionConvLayer(64, 2, 32, [256, 256], 128)
        # [B, 16, 512+3]
        self.graph_pooling4 = GraphAttentionConvLayer(16, 2, 32, [512, 512], 256)
        
        # FeaturePropagationLayer(in_channel, mlp)
        # in_channel = 256 + 512
        self.fp4 = FeaturePropagationLayer(256 + 512, [256, 256])
        # in_channel = 128 + 256
        self.fp3 = FeaturePropagationLayer(128 + 256, [256, 128])
        # in_channel = 32 + 128
        self.fp2 = FeaturePropagationLayer(32 + 128, [128, 32])
        # in_channel = 6 + 32
        self.fp1 = FeaturePropagationLayer(6 + 32, [32, 32])

        self.GAC_Layer = GraphAttentionConvLayer(4096, 2, 3, [32, 64], 32)

        self.conv1d = nn.Conv1d(64+3, 32, 1)
        self.bn = nn.BatchNorm1d(32)

        self.fc = nn.Linear(32, num_classes)

    def forward(self, points):
        # [B, 1024, 32+3]
        layer1_points = self.graph_pooling1(points)
        # [B, 256, 128+3]
        layer2_points = self.graph_pooling2(layer1_points)
        # [B, 64, 256+3]
        layer3_points = self.graph_pooling3(layer2_points)
        # [B, 16, 512+3]
        layer4_points = self.graph_pooling4(layer3_points)

        # [B, 64, 256+3]
        layer3_points = self.fp4(layer3_points, layer4_points)
        # [B, 256, 128+3]
        layer2_points = self.fp3(layer2_points, layer3_points)
        # [B, 1024, 32+3]
        layer1_points = self.fp2(layer1_points, layer2_points)
        # [B, 4096, 32+3]
        layer0_points = self.fp1(points, layer1_points)
        # [B, 4096, 64+3]
        layer0_points = self.GAC_Layer(layer0_points)
        # [B, 32, 4096]
        layer0_points = F.relu(self.bn(self.conv1d(layer0_points.permute(0,2,1))))
        # [B, 4096, num_classes]
        layer0_points = self.fc(layer0_points.permute(0,2,1))
        layer0_points = layer0_points.softmax(-1)
        
        return layer0_points

In [79]:
net = GAC_Net(13)
gp = net(points)

In [80]:
class_id = gp.max(-1)[1]
class_id.shape

torch.Size([4, 4096])

In [1]:
import torch